In [ ]:
//var Collection = require('dstools').Collection;
//var data = Collection().loadCSV('data.csv');
// var data = Collection([{field1:2,field2:4},{field1:3,field2:5},{field1:2,field2:4}
//                        ,{field1:4,field2:4},{field1:4,field2:4},{field1:2,field2:4}]);

//var data = Collection().loadCSV('heart.csv');
//var lib = require('../lib')


var fs = require('fs');
//var _init = fs.readFileSync('init.js').toString();
//eval(_init);
//setupD3();


In [36]:
function cleanData(columns, array) {
	if( !hasNoEmptyElem(columns) ) throw new Error("Invalid frame column");
	let n = columns.length;
	return [columns, array.map(_cleanData).filter(s => s)]

}

function hasNoEmptyElem(arr, v) {
	if( arguments.length === 1 ){
		for(let i=0; i< arr.length; i++) 
			if(!arr[i]) return undefined;
	} else {
		for(let i=0; i< arr.length; i++) 
			if(arr[i] === v ) return undefined;
	}
	return arr;
}

function arrRemUndef(arr) {
	for(let i=0; i< arr.length; i++) 
		if(arr[i] === undefined) return arr.filter(v => v !== undefined);
	return arr;
}

function _cleanData(n) {
	return arr => {
		arr = arrRemUndef(arr);
		if(n !== arr.length) {
			if(n > arr.length) {
				arr = arr.slice(0);
				while(arr.length < n) arr.push('');
			} else {
	            if(n === arr.length) return arr;
				if(n < arr.length) return arr.slice(0,n);
			}
		}
		return hasNoEmptyElem(arr, undefined);
	}
}

[hasNoEmptyElem(['a','1', 'b']), 
 hasNoEmptyElem(['a','', 'b']), 
 hasNoEmptyElem(['a','', 'b'], undefined), 
 hasNoEmptyElem(['a',undefined, 'b']),
  hasNoEmptyElem(['a',undefined, 'b'], '')
 ]

[
  [ 'a', '1', 'b' ],
  undefined,
  [ 'a', '', 'b' ],
  undefined,
  [ 'a', undefined, 'b' ]
]

In [37]:
[[arrRemUndef(['a','1', 'b']), 
 arrRemUndef(['a','', 'b']), 
 arrRemUndef(['a',undefined, 'b']),
 arrRemUndef([undefined,undefined, 'b'])
],
[_cleanData(3)(['a','1', 'b']), 
 _cleanData(3)(['a','', 'b']),
 _cleanData(2)(['a','', 'b']), 
 _cleanData(2)(['a',undefined, 'b']),
 _cleanData(3)(['5', '7', '8', undefined,undefined, 'b'])
]
]

[
  [ [ 'a', '1', 'b' ], [ 'a', '', 'b' ], [ 'a', 'b' ], [ 'b' ] ],
  [
    [ 'a', '1', 'b' ],
    [ 'a', '', 'b' ],
    [ 'a', '' ],
    [ 'a', 'b' ],
    [ '5', '7', '8' ]
  ]
]

In [14]:
var fs = require('fs');
var str = fs.readFileSync('../dist/bundle.js');
var zlib = require('zlib');
eval(str.toString('utf8'));
var {Frame, frameFromBuffer, csvLine, tsvLine, gb} = exports;
function readZipData(fileName) {
  const buffer = fs.readFileSync(fileName); // Read from file
  // console.log("bytes read", buffer.length)
  return zlib.gunzipSync(buffer);
}
 
Object.keys(exports)

[
  'Accum',            'FLATTEN',           'Frame',
  'MultiDict',        'MultiDictSet',      'StrDict',
  'SumDict',          '__ARRAY__',         '__DICT__',
  '__FRAME__',        '__STRING__',        'accStr',
  'access',           'arrCMP',            'arrConcat',
  'arrCount',         'arrCountVal',       'arrCurry',
  'arrDedup',         'arrDiff',           'arrDistinct',
  'arrEQ',            'arrHash',           'arrIntersect',
  'arrMax',           'arrMean',           'arrMin',
  'arrOf',            'arrProd',           'arrRemove',
  'arrSplit',         'arrSum',            'arrToCol',
  'arrUnion',         'arrZip',            'asSortedSet',
  'countCh',          'csvLine',           'dataSplit',
  'dedupSortedArr',   'dictToZip',         'dictToZipB',
  'filterDict',       'filterIX',          'findIndexOf',
  'flatten',          'frameCount',        'frameFromBuffer',
  'frameSum',         'frameToString',     'frameWithIndex',
  'fromKeyValueList', 'fromKeyValueListV

In [18]:
var request = require('request');
var WORLD_DATA_SRC = 'https://covid.ourworldindata.org/data/full_data.csv';
request.get(WORLD_DATA_SRC, function (error, response, body) {
    if (!error && response.statusCode == 200) {
        var csv = body;
        frame = frameFromBuffer(csv.toString(),'',csvLine);
        frame.data = frame.data.slice(0, frame.length-2)
    }
});
0

0

In [24]:
var {Frame, frameFromBuffer, csvLine, tsvLine, gb} = exports;
frame.groupBy(['location',gb.count('date','count')]).sort(['-count'])


Ix,location,count
0,China,57
1,Japan,57
2,South Korea,57
3,Thailand,57
4,World,56
5,United States,55
6,Singapore,54
7,Vietnam,54
8,Australia,53
9,France,53


In [2]:
var s = readZipData('../data/reference/all_icd10_codes.tsv.gz').toString('utf8');
var {Frame, frameFromBuffer, tsvLine} = exports;
var frame = frameFromBuffer(s,'',tsvLine);
//frame
frame.groupBy(['TYPE'])

Ix,TYPE
0,CM
1,PCS


In [ ]:
var ff = require('../lib');
//var Frame = ff.Frame;

In [ ]:
function doN(n, fn) {
    return () =>{
        var res;
        for(let i=0;i<n; i++) { res = fn(res); }
        return res;
    }
}

In [ ]:
var ICD10 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var A = r => r.ICD10[0] === 'A';
ICD10.slice(0,20);

In [ ]:
 var _ = timeOf(doN(1, () => ICD10.data.filter(r => r[0][0] === 'A' )));


In [ ]:
function filter(fr,fn) {
    let data = fr.data;
    let res = [];
    for(let i=0; i<data.length; i++) {
        let d = data[i];
        if(fn(d,i)) res.push(d);
    }
    return res;
}
var _ = timeOf(doN(1, () => filter(ICD10,r => r[0][0] === 'A' )));


ICD10.groupBy(['TYPE'])

In [ ]:
var exp = require('../dist/bundle');
exp

In [ ]:
function getAllMethods(obj) {
    let props = []

    do {
        const l = Object.getOwnPropertyNames(obj)
            .concat(Object.getOwnPropertySymbols(obj).map(s => s.toString()))
            .sort()
            .filter((p, i, arr) =>
                typeof obj[p] === 'function' &&  //only the methods
                p !== 'constructor' &&           //not the constructor
                (i == 0 || p !== arr[i - 1]) &&  //not overriding in this prototype
                props.indexOf(p) === -1          //not overridden in a child
            )
        props = props.concat(l)
    }
    while (
        (obj = Object.getPrototypeOf(obj)) &&   //walk-up the prototype chain
        Object.getPrototypeOf(obj)              //not the the Object prototype methods (hasOwnProperty, etc...)
    )

    return props
}
getAllMethods( new exp.Frame());


In [ ]:
var exp = require('../lib');
function PICK(name,...args) {
    return o => o[name](...args);
}
Object.keys(exp).filter(s => s.match(/__(.*)__/))
    .map(s => s.replace(/__(.*)__/, "$1"))
    .map(PICK('toLowerCase')).map(n => require('../lib/'+n));


In [ ]:
var CNT = 1
var myData = timeOf(doN(CNT, () =>ICD10.filter(r => !r.ICD10.match(/^A/) ).project(['ICD10'])))
var __ =   timeOf(doN(CNT, () =>ICD10.filter(r => !r.ICD10.startsWith('A') ).project(['ICD10'])))
var ___ =  timeOf(doN(CNT, () =>ICD10.filter(NOT(A)).project(['ICD10'])))


__

In [ ]:
var getMethods = (obj) => Object.getOwnPropertyNames(obj).filter(item => typeof obj[item] === 'function')
CNT=1;
console.log('len', ICD10.length)
var x =timeOf(doN(CNT,() => ICD10.innerJoin(myData, [...ICD10.columns.map(x => '1.'+x)], 'ICD10')))//.slice(10,20)
timeOf(() =>ICD10.project(["T", 'TYPE', 'ICD10'], {T: r => r.ICD10[0] },true)
    .groupBy(['T', 'TYPE', gb.count('ICD10', 'CNT')])
    .sort(['-CNT']));
//gb

In [ ]:
function renameColumns(frame,map) {
	return frame.columns.map(c => map[c]?map[c]:c);

}



In [ ]:
var NAMES = new Frame([['PCS', "Procedure"], ['CM', "Diagnostic"],["???", "Unknown"]], ["TYPE", "DESC"], "ICD-TYPE");
var VALUES = new Frame([['PCS', "Institution"], ['CM', "Any"],["UNK", "Unknown"]], ["TYPE", "LOC"], "ICD-TYPE");



keys(gb)

In [ ]:
NAMES.leftJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC=TYPE'}), "TYPE")
      .groupBy(['TYPE', gb.count('TYPE','CNT')])//.groupBy([gb.min('CNT')])

In [ ]:
NAMES.leftJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE')

In [ ]:
NAMES.outerJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE').sort(["DESC", "TYPE"])

In [ ]:
NAMES.innerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

In [ ]:
NAMES.outerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

In [ ]:
//var {Frame} = require('../dist/bundle.js');

In [ ]:
var C = class {
    constructor(a) {
        this.value = a;
    }
    
    
};

var c = new C(1);
new c.constructor(2);
function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

In [ ]:
var la = require('../lib/array')

In [ ]:
function test(n) {
    var a = la.newArray(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test0(n) {
    var a = new Int32Array(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test2(n) {
    var a = [];
    for(let i=0; i<n; i++) a.push(i);
    return a;
}

function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

[timeOf(() => la.arrSum(test(10000000))),
timeOf(() => la.arrSum(test0(10000000)))];
var a = test0(10000000);
var b = test2(10000000);


In [ ]:
[timeOf(() => {for(i=0; i<100; i++)la.arrSum(b)}),
timeOf(() => {for(i=0; i<100; i++)la.arrSum(a)})]

In [ ]:
//var {Frame} = require('../lib/frame')

In [ ]:
var f = new Frame( [[1,2],[3,4]],['a','b'],'test');
ArrayBuffer.isView(new Int32Array(10))

In [ ]:
f.sort(['-b'])

In [ ]:
fs.readdirSync('../data/reference');
var ICD9 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var HCPCS = readZipFrame('../data/reference/hcpcs.tsv.gz')
HCPCS = HCPCS.project(['TYPE', ...HCPCS.columns], {TYPE: r => r.CODE.substr(0,2)},true);
HCPCS.groupBy(['TYPE', gb.count('TYPE', 'COUNT')]).sort(['-COUNT'])

In [ ]:
HCPCS.filter(r => r.TYPE === '90').groupBy(['TYPE', gb.count('TYPE', 'COUNT')])


In [ ]:
$$.html(`<script>
console.log(define.toString())
</script>`)


In [ ]:
var CFrame = class {
    constructor(data, columns, isColumn=true) {
        this.columns = columns;
        let len = isColumn? data[0].length : data.length;
        this.length = len;
        if(!column) {
           let rLn = this.columns.length;
           let aData = arrayOf(() => new Array(len), rLn);
           this.data = aData;
           for(let i=0; i<len; i++) {
               let row = data[i];
               let j = 0;
               for(; j< rLn-7; j+= 8) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
                   aData[j+4][i] = row[j+4];
                   aData[j+5][i] = row[j+5];
                   aData[j+6][i] = row[j+6];
                   aData[j+7][i] = row[j+7];
               }
               for(; j< rLn-3; j+= 4) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
               }
               for(; j< rLn; j++) {
                   aData[j][i] = row[j]
               }
           }
       }
    }

    /**
     * old Column array
    _colMapper(mapArray, baseData) {
        
    }*/
}

#Adam optimizer

In [ ]:
var result, result1;
var http = require('https');
http.get("https://raw.githubusercontent.com/cmusam/fortune500/master/csv/fortune500-2017.csv",
         (resp) =>{ 
  let data = '';

  // A chunk of data has been recieved.
  resp.on('data', (chunk) => {
    data += chunk;
  });

  // The whole response has been received. Print out the result.
  resp.on('end', () => {
    result1 = data.replace(/\r/g, '');
    result =  result1 .split('\n').map(s => s.split(','));
  });
});
console.log(result);

In [ ]:
var fortune = timeOf(() => new Frame(result.slice(1),result[0],'Fortune')
                     .sort(["-profit ($ millions)"]))
//fortune.filter(r => console.log(r.__data[1]))
fortune.filterRaw(r => r[1] === undefined)

In [ ]:
fortune.filter(r => { console.log(r, r.company[0]); })


In [ ]:
var fortune = timeOf(() => ff.frameFromBuffer(result1,'fortune',ff.csvLine).sort(["-profit ($ millions)"]))
fortune.filter(r => {  var n = r.company;  console.log(r.company); return n.substr(0,1) === 'A'})
